# 0. Load data

In [471]:

import tensorflow_hub as hub
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w

from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.feature import HashingTF, IDF,  Tokenizer, RegexTokenizer, CountVectorizer, StopWordsRemover, NGram, Normalizer, VectorAssembler, Word2Vec, Word2VecModel, PCA
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.clustering import LDA
from pyspark.mllib.linalg import Vectors, VectorUDT

spark = SparkSession.builder.getOrCreate()

In [472]:
df = spark.read.csv("data/X2.csv", header=True)
df.toPandas()

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title
0,www.softwarecity.ca//737,Lenovo,Intel. i5-3320M,i5-3320M,Dual-core ( 2 Core ). Core i5,2.60 GHz,None,DDR3 SDRAM. DDR3-1600/PC3-12800. DDR3 SDRAM,DDR3-1600/PC3-12800,320 GB,None,1.80 kg,None,"""Lenovo Thinkpad X230 34352jf Tablet Pc - 12.5..."
1,www.isupplyhub.com//1256,Acer,1.6 GHz Intel Core i5-4200U. Intel Core I5,None,1.6 GHz Intel Core i5-4200U,1.6 GHz Intel Core i5-4200U,8 GB DDR3L SDRAM,DDR3 SDRAM. 8 GB DDR3L SDRAM,None,500 GB mechanical_hard_drive,None,4.8 pounds,15.02 x 10.08 x 0.90 inches,Amazon.com : Acer Aspire V7-582PG-6479 15.6-In...
2,www.isupplyhub.com//326,Acer,1.6 GHz Intel Core i5. Intel Core I5,None,1.6 GHz Intel Core i5,1.6 GHz Intel Core i5,4 GB DDR3-SDRAM,DDR3 SDRAM. 4 GB DDR3-SDRAM,None,500 GB mechanical_hard_drive,None,5.2 pounds,15.02 x 10.08 x 1 inches,Amazon.com : Acer Aspire E1-572-6870 15.6 Inch...
3,www.isupplyhub.com//821,HP,None,None,None,None,4 GB SDRAM DDR3,DDR3 SDRAM. 4 GB SDRAM DDR3,None,500 GB,None,4.8 pounds,15.18 x 0.89 x 10.16 inches,"""Amazon.com : 15.6"""" HP 15-f009wm Amd Dual-Cor..."
4,www.isupplyhub.com//157,Asus,1.7 GHz Core i5-3317U. Intel,None,1.7 GHz Core i5-3317U,1.7 GHz Core i5-3317U,4 GB DDR3,DDR3 SDRAM. 4 GB DDR3,None,256 MB,None,2.9 pounds,8.80 x 0.70 x 12.80 inches,Amazon.com : ASUS UX31A-XB52 13.3-Inch Ultrabo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,Lenovo ThinkPad X230 2320 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,4 GB DDR3 Slots Qty 2 Empty Slots 1 Max RAM Su...,180 GB SSD. 180 GB SSD. Lenovo ThinkPad X230 2...,180 GB SSD. 180 GB SSD,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2320 - 12.5"""" - Core i5 ..."
339,www.vology.com//823,Lenovo ThinkPad X230 2325 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,4 GB DDR3 Slots Qty 2 Max RAM Supported 16 GB ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2325 - 12.5"""" - Core i5 ..."
340,www.vology.com//2723,Lenovo ThinkPad X230 Tablet 3438 - 12.5 '' - C...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm. ...,500 GB HDD / 7200 rpm. 500 GB HDD / 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,"""Lenovo ThinkPad X230 Tablet 3438 - 12.5"""" - C..."
341,www.vology.com//1349,Lenovo ThinkPad X230 2324 - 12.5 '' - Core i5 ...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Intel Core i5 ( 3rd Gen ) 3320M / 2.6 GHz. Int...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,Form Factor SO DIMM 204-pin Technology DDR3 SD...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm. ...,320 GB HDD / 7200 rpm. 320 GB HDD / 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,"""Lenovo ThinkPad X230 2324 - 12.5"""" - Core i

# 0. Data cleaning

In [473]:
for c in df.columns:
#set everything to lowercase
    df = df.withColumn(c, f.lower(f.col(c)))

#extract brand or infer from title
df = df.withColumn('brand', f.regexp_extract('brand', "^(\w+)", 0))
computer_brands = ['(lenovo', 'acer', 'hp', 'dell', 'asus', 'samsung', 'huawei', 'surface', 'apple)']
computer_brands_pattern = '|'.join(computer_brands)
df = df.withColumn('brand', f.when( f.regexp_extract('title', computer_brands_pattern, 0)!='', f.regexp_extract('title', computer_brands_pattern, 0))\
                   .otherwise(df.brand))
#exctract cpu_brand and infer type if intel
cpu_brands = ['(intel', 'apple', 'amd', 'nvidia', 'arm)']
cpu_pattern = '|'.join(cpu_brands)
df = df.withColumn('cpu_model', f.when( (f.regexp_extract('cpu_brand','intel',0)!='') & (df.cpu_model == '') ,\
                                        f.regexp_extract('cpu_type', '((i\d)|pentium|celeron)', 0))\
                   .otherwise(df.cpu_model))
df = df.withColumn('cpu_brand', f.when(f.regexp_extract('cpu_brand', cpu_pattern, 0) != '', f.regexp_extract('cpu_brand', cpu_pattern, 1))\
                                       .otherwise(f.regexp_extract('title', cpu_pattern, 0)))
#cpu type inferring not working

# 1. Blocking

In [474]:
"""UTILITIES"""

"""Returns the df with tokenized columns with stopwords removed"""
def tokenize(df, string_cols):
  output = df
  stopW = ['softwarecity', 'amazon', 'com', 'computers', 'computer', 'accessories', 'laptop', 'notebook', 'kg', 'inch', 'processor', 'memory','gb', 'ram', 'hdd', 'ssd', 'cpu', 'display', 'hz', 'ghz', 'tb', 'cache', 'ddram', 'dram', 'pro', 'slot', 'hd']
  for c in string_cols:
    output = output.withColumn('temp', f.coalesce(f.col(c), f.lower(c), f.lit('')))
    tokenizer = RegexTokenizer(inputCol='temp', outputCol=c+"_tokens", pattern = "\\W")
    remover = StopWordsRemover(inputCol=c+"_tokens", outputCol=c+"_swRemoved", stopWords=stopW)
    output = tokenizer.transform(output)

    filter_alnum = f.udf(lambda l : [t for t in l[:10] if t.isalpha()], t.ArrayType(t.StringType()))
    output = output.withColumn(c+'_tokens', filter_alnum(f.col(c+"_tokens")))

    output.select(c+'_tokens').show()

    output = remover.transform(output)\
      .drop('temp', c+"_tokens")
    # output has c+swRemoved columns
    output.select(c+'_swRemoved').show()

  return output

"""Selects the n top keywords from the tokens in each row from a vocab(from one col) using TFIDF score"""
def top_kw_from_tfidf(vocab, n=3):
  @f.udf(returnType=t.ArrayType(t.StringType()))
  def _(arr):
    inds = arr.indices
    vals = arr.values
    top_inds = vals.argsort()[-n:][::-1]
    top_keys = inds[top_inds]
    output = []

    for k in top_keys:
      kw = vocab.value[k]
      output.append(kw)

    return output
  return _

"""Selects top n tokens using TFIDF score for each token_cols using above func"""
def generate_blocking_keys(df, token_cols, min_freq=1):
    """Pipeline:
            1 - CountVectorizer -> TF
            2 - IDF
            3 - LDA
    """
    cv = CountVectorizer(inputCol='title_swRemoved', outputCol="rawFeatures")
    cvmodel = cv.fit(df)
    df_vect = cvmodel.transform(df)

    idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=min_freq)
    idfModel = idf.fit(df_vect)
    df_idf= idfModel.transform(df_vect)

    normalizer = Normalizer(p=2.0, inputCol='features', outputCol='tfidf')
    output = normalizer.transform(df_idf)

    lda = LDA(k=10, maxIter=200, featuresCol='tfidf')
    lda_model = lda.fit(output)
    vocab = cvmodel.vocabulary

    def get_words(token_list):
        return [vocab[token_id] for token_id in token_list]

    udf_to_words = f.udf(get_words, t.ArrayType(t.StringType()))

    topics = lda_model.describeTopics(2).withColumn('topicWords', udf_to_words(f.col('termIndices')))\
        .show()
    output = lda_model.transform(output)
    output.select('topicDistribution').show()

    #returns the index of the topic for which the distr is higher
    def get_key(topicDistr):
        max_i = 0
        max = 0.0
        i = 0
        for t in topicDistr:
            if t >= max:
                max = t
                max_i = i
            i += 1
        return max_i

    udf_get_key = f.udf(get_key, t.IntegerType())
    output = output.withColumn("blocking_key", udf_get_key(f.col("topicDistribution")))
    output.select("blocking_key").show()
    return output

"""Use universal sentence encoder from tensorflow_hub"""
MODEL = None
def get_model_magic():
  global MODEL
  if MODEL is None:
      MODEL = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  return MODEL

@f.udf(returnType=VectorUDT())
def encode_sentence(x):
  model = get_model_magic()
  emb = model([x]).numpy()[0]
  return Vectors.dense(emb)

In [475]:
columns = ['title']
blocking_df = tokenize(df, columns)

+--------------------+
|        title_tokens|
+--------------------+
|[lenovo, thinkpad...|
|[amazon, com, ace...|
|[amazon, com, ace...|
|[amazon, com, hp,...|
|[amazon, com, asu...|
|[amazon, com, len...|
|[amazon, com, ace...|
|[amazon, com, ace...|
|[amazon, com, len...|
|[amazon, com, ace...|
|[amazon, com, del...|
|[amazon, com, del...|
|[amazon, com, del...|
|[amazon, com, ace...|
|[amazon, com, ace...|
|[amazon, com, ace...|
|[amazon, com, len...|
|[amazon, com, del...|
|[amazon, com, ace...|
|[amazon, com, ace...|
+--------------------+
only showing top 20 rows

+--------------------+
|     title_swRemoved|
+--------------------+
|[lenovo, thinkpad...|
|      [acer, aspire]|
|      [acer, aspire]|
|[hp, amd, dual, c...|
|   [asus, ultrabook]|
|[lenovo, thinkpad...|
|  [acer, aspire, nx]|
| [acer, aspire, led]|
|[lenovo, thinkpad...|
|      [acer, aspire]|
|    [dell, inspiron]|
|[dell, inspiron, ...|
|[dell, inspiron, ...|
|      [acer, aspire]|
|      [acer, aspire]|
|      [

In [476]:
blocking_df = generate_blocking_keys(blocking_df, [c+'_swRemoved' for c in columns])

+-----+-----------+--------------------+--------------------+
|topic|termIndices|         termWeights|          topicWords|
+-----+-----------+--------------------+--------------------+
|    0|     [4, 3]|[0.43074674760303...|   [carbon, windows]|
|    1|    [7, 10]|[0.36425412266603...|     [tablet, touch]|
|    2|   [32, 31]|[0.20629053431942...|  [edition, special]|
|    3|   [16, 15]|[0.23161261610483...|    [dell, inspiron]|
|    4|     [3, 2]|[0.37126107861686...| [windows, thinkpad]|
|    5|     [5, 6]|[0.27965031400224...|      [acer, aspire]|
|    6|     [9, 8]|[0.22557422156027...|     [elitebook, hp]|
|    7|   [51, 49]|[0.09749765035982...|   [with, ultrabase]|
|    8|    [53, 2]|[0.14522459789944...|[ultraportable, t...|
|    9|   [25, 23]|[0.10418307459598...| [high, performance]|
+-----+-----------+--------------------+--------------------+

+--------------------+
|   topicDistribution|
+--------------------+
|[0.03698025391298...|
|[0.04610860718130...|
|[0.046108607181

In [477]:
blocking_df.groupby('blocking_key').count().show()


+------------+-----+
|blocking_key|count|
+------------+-----+
|           1|   75|
|           6|   35|
|           3|   12|
|           5|   57|
|           9|    5|
|           4|  102|
|           8|    1|
|           7|    1|
|           2|    3|
|           0|   52|
+------------+-----+

